<a href="https://colab.research.google.com/github/cifcs2/Sentiment-Analysis-of-Former-Presidents-SONA-using-NLP/blob/main/firstspeechesfortrainingsoon.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [ ]:
from __future__ import print_function
#!pip install plotly_express
#!pip install transformers
import ipywidgets as widgets
import matplotlib as mpl
import pandas as pd
import plotly
import plotly_express as px
import plotly.graph_objects as go
import numpy as np
import re

from transformers import pipeline

mpl.rcParams["figure.dpi"] = 300
%matplotlib inline
%config InlineBackend.figure_format ='retina'

Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
Looking in indexes: https://pypi.org/simple, https://us-python.pkg.dev/colab-wheels/public/simple/
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 6.8/6.8 MB 54.9 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 199.8/199.8 KB 26.3 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 7.6/7.6 MB 107.1 MB/s eta 0:00:00


**data scraping**

In [ ]:
# Web scraping, pickle imports
import requests
from bs4 import BeautifulSoup
import pickle

# Scrapes transcript data from scrapsfromtheloft.com
def url_to_transcript(url):
    page = requests.get(url).text
    soup = BeautifulSoup(page, "lxml")
    text = [p.text for p in soup.find(class_="entry-content").find_all('p')]
    print(url)
    return text

# URLs of transcripts in scope
urls = ['https://www.officialgazette.gov.ph/1935/11/25/manuel-l-quezon-first-state-of-the-nation-address-november-25-1935/', 
'https://www.officialgazette.gov.ph/1936/06/16/manuel-l-quezon-second-state-of-the-nation-address-june-16-1936/', 
'https://www.officialgazette.gov.ph/1937/10/18/manuel-l-quezon-third-state-of-the-nation-address-october-18-1937/', 
'https://www.officialgazette.gov.ph/1938/01/24/manuel-l-quezon-fourth-state-of-the-nation-address-january-24-1938/', 
'https://www.officialgazette.gov.ph/1939/01/24/manuel-l-quezon-fifth-state-of-the-nation-address-january-24-1939/', 
'https://www.officialgazette.gov.ph/1940/01/22/manuel-l-quezon-sixth-state-of-the-nation-address-january-22-1940/', 
'https://www.officialgazette.gov.ph/1941/01/31/manuel-l-quezon-seventh-state-of-the-nation-address-january-31-1941/', 
'https://www.officialgazette.gov.ph/1945/06/09/sergio-osmena-state-of-the-nation-address-june-9-1945/', 
'https://www.officialgazette.gov.ph/1946/06/03/president-roxas-on-first-state-of-the-nation-address-june-3-1946/', 
'https://www.officialgazette.gov.ph/1947/01/27/manuel-roxas-second-state-of-the-nation-address-january-27-1947/', 
'https://www.officialgazette.gov.ph/1948/01/26/manuel-roxas-the-nation-on-the-road-to-prosperity-third-state-of-the-nation-address-january-26-1948/', 
'https://www.officialgazette.gov.ph/1949/01/24/state-of-the-nation-message-of-president-quirino-to-the-joint-session-of-the-congress-of-the-philippines/', 
'https://www.officialgazette.gov.ph/1950/01/23/elpidio-quirino-second-state-of-the-nation-address-january-23-1950/', 
'https://www.officialgazette.gov.ph/1951/01/22/elpidio-quirino-third-state-of-the-nation-address-january-22-1951/', 
'https://www.officialgazette.gov.ph/1952/01/28/elpidio-quirino-fourth-state-of-the-nation-address-january-28-1952/', 
'https://www.officialgazette.gov.ph/1953/01/26/elpidio-quirino-fifth-state-of-the-nation-address-january-26-1953/', 
'https://www.officialgazette.gov.ph/1954/01/25/ramon-magsaysay-first-state-of-the-nation-address-january-25-1954/', 
'https://www.officialgazette.gov.ph/1955/01/24/ramon-magsaysay-second-state-of-the-nation-address-january-24-1955/', 
'https://www.officialgazette.gov.ph/1956/01/23/ramon-magsaysay-third-state-of-the-nation-address-january-23-1956/', 
'https://www.officialgazette.gov.ph/1957/01/28/ramon-magsaysay-fourth-state-of-the-nation-address-january-28-1957/', 
'https://www.officialgazette.gov.ph/1958/01/27/message-of-president-carlos-p-garcia-on-his-first-state-of-the-nation-address/', 
'https://www.officialgazette.gov.ph/1959/01/26/carlos-p-garcia-second-state-of-the-nation-address-january-26-1959/', 
'https://www.officialgazette.gov.ph/1960/01/25/carlos-p-garcia-third-state-of-the-nation-address-january-25-1960/', 
'https://www.officialgazette.gov.ph/1961/01/23/carlos-p-garcia-fourth-state-of-the-nation-address-january-23-1961/', 
'https://www.officialgazette.gov.ph/1962/01/22/diosdado-macapagal-first-state-of-the-nation-address-january-22-1962/', 
'https://www.officialgazette.gov.ph/1963/01/28/diosdado-macapagal-second-state-of-the-nation-address-january-28-1963/', 
'https://www.officialgazette.gov.ph/1964/01/27/diosdado-macapagal-third-state-of-the-nation-address-january-27-1964/', 
'https://www.officialgazette.gov.ph/1965/01/25/diosdado-macapagal-fourth-state-of-the-nation-address-january-25-1965/', 
'https://www.officialgazette.gov.ph/1966/01/24/ferdinand-e-marcos-first-state-of-the-nation-address-january-24-1966/', 
'https://www.officialgazette.gov.ph/1967/01/23/ferdinand-e-marcos-second-state-of-the-nation-address-january-23-1967/', 
'https://www.officialgazette.gov.ph/1968/01/22/ferdinand-e-marcos-third-state-of-the-nation-address-january-22-1968/', 
'https://www.officialgazette.gov.ph/1969/01/27/ferdinand-e-marcos-fourth-state-of-the-nation-address-january-27-1969/', 
'https://www.officialgazette.gov.ph/1970/01/26/ferdinand-e-marcos-fifth-state-of-the-nation-address-january-26-1970/', 
'https://www.officialgazette.gov.ph/1971/01/25/ferdinand-e-marcos-sixth-state-of-the-nation-address-january-25-1971-2/', 
'https://www.officialgazette.gov.ph/1972/01/24/ferdinand-e-marcos-seventh-state-of-the-nation-address-january-24-1972/', 
'https://www.officialgazette.gov.ph/1973/09/21/ferdinand-e-marcos-eighth-state-of-the-nation-address-september-21-1973/', 
'https://www.officialgazette.gov.ph/1974/09/21/ferdinand-e-marcos-ninth-state-of-the-nation-address-september-21-1974/', 
'https://www.officialgazette.gov.ph/1975/09/19/ferdinand-e-marcos-tenth-state-of-the-nation-address-september-19-1975/', 
'https://www.officialgazette.gov.ph/1976/09/21/ferdinand-e-marcos-eleventh-state-of-the-nation-address-september-21-1976/', 
'https://www.officialgazette.gov.ph/1977/09/21/ferdinand-e-marcos-twelfth-state-of-the-nation-address-september-21-1977/', 
'https://www.officialgazette.gov.ph/1978/06/12/ferdinand-e-marcos-thirteenth-state-of-the-nation-address-june-12-1978/', 
'https://www.officialgazette.gov.ph/1979/07/23/ferdinand-e-marcos-fourteenth-state-of-the-nation-address-july-23-1979/', 
'https://www.officialgazette.gov.ph/1980/07/28/ferdinand-e-marcos-fifteenth-state-of-the-nation-address-july-28-1980/', 
'https://www.officialgazette.gov.ph/1981/07/27/ferdinand-e-marcos-sixteenth-state-of-the-nation-address-july-27-1981/', 
'https://www.officialgazette.gov.ph/1982/07/26/ferdinand-e-marcos-seventeenth-state-of-the-nation-adress-july-26-1982/', 
'https://www.officialgazette.gov.ph/1983/01/17/ferdinand-e-marcos-eighteenth-state-of-the-nation-address-january-17-1983/', 
'https://www.officialgazette.gov.ph/1984/07/23/ferdinand-e-marcos-nineteenth-state-of-the-nation-address-july-23-1984/', 
'https://www.officialgazette.gov.ph/1985/07/22/ferdinand-e-marcos-twentieth-state-of-the-nation-address-july-22-1985/', 
'https://www.officialgazette.gov.ph/1987/07/27/corazon-c-aquino-first-state-of-the-nation-address-july-27-1987/', 
'https://www.officialgazette.gov.ph/1988/07/25/corazon-c-aquino-second-state-of-the-nation-address-july-25-1988/', 
'https://www.officialgazette.gov.ph/1989/07/24/corazon-c-aquino-third-state-of-the-nation-address-july-24-1989/', 
'https://www.officialgazette.gov.ph/1990/07/23/corazon-c-aquino-fourth-state-of-the-nation-address-july-23-1990/', 
'https://www.officialgazette.gov.ph/1991/07/22/corazon-c-aquino-fifth-state-of-the-nation-address-july-22-1991/', 
'https://www.officialgazette.gov.ph/1992/07/27/fidel-v-ramos-first-state-of-the-nation-address-july-27-1992/', 
'https://www.officialgazette.gov.ph/1993/07/26/fidel-v-ramos-second-state-of-the-nation-address-july-26-1993/', 
'https://www.officialgazette.gov.ph/1994/07/25/fidel-v-ramos-third-state-of-the-nation-address-july-25-1994/', 
'https://www.officialgazette.gov.ph/1995/07/24/fidel-v-ramos-fourth-state-of-the-nation-address-july-24-1995/', 
'https://www.officialgazette.gov.ph/1996/07/22/fidel-v-ramos-fifth-state-of-the-nation-address-july-22-1996/', 
'https://www.officialgazette.gov.ph/1997/07/28/fidel-v-ramos-sixth-state-of-the-nation-address-july-28-1997/' ]       

# President names
presidents = ['Quezon-1935', 'Quezon-1936', 'Quezon-1937', 'Quezon-1938', 'Quezon-1939',
              'Quezon-1940', 'Quezon-1941', 
              'Osmeña-1945', 
              'Roxas-1946', 'Roxas-1947','Roxas-1948', 
              'Quirino-1949', 'Quirino-1950', 'Quirino-1951', 'Quirino-1952','Quirino-1953', 
              'Magsaysay-1954', 'Magsaysay-1955', 'Magsaysay-1956', 'Magsaysay-1957',
              'Garcia-1958', 'Garcia-1959', 'Garcia-1960', 'Garcia-1961', 
              'DiosdadoMacapagal-1962','DiosdadoMacapagal-1963', 'DiosdadoMacapagal-1964', 'DiosdadoMacapagal-1965', 
              'Marcos-1966','Marcos-1967', 'Marcos-1968', 'Marcos-1969', 'Marcos-1970', 'Marcos-1971',
              'Marcos-1972', 'Marcos-1973', 'Marcos-1974', 'Marcos-1975', 'Marcos-1976',
              'Marcos-1977', 'Marcos-1978', 'Marcos-1979', 'Marcos-1980', 'Marcos-1981',
              'Marcos-1982', 'Marcos-1983', 'Marcos-1984', 'Marcos-1985', 
              'CorazonCAquino-1987', 'CorazonCAquino-1988', 'CorazonCAquino-1989', 'CorazonCAquino-1990', 'CorazonCAquino-1991',
              'Ramos-1992', 'Ramos-1993', 'Ramos-1994', 'Ramos-1995', 'Ramos-1996', 'Ramos-1997']

In [ ]:
# # Actually request transcripts (takes a few minutes to run)
transcripts = [url_to_transcript(u) for u in urls]

https://www.officialgazette.gov.ph/1935/11/25/manuel-l-quezon-first-state-of-the-nation-address-november-25-1935/
https://www.officialgazette.gov.ph/1936/06/16/manuel-l-quezon-second-state-of-the-nation-address-june-16-1936/
https://www.officialgazette.gov.ph/1937/10/18/manuel-l-quezon-third-state-of-the-nation-address-october-18-1937/
https://www.officialgazette.gov.ph/1938/01/24/manuel-l-quezon-fourth-state-of-the-nation-address-january-24-1938/
https://www.officialgazette.gov.ph/1939/01/24/manuel-l-quezon-fifth-state-of-the-nation-address-january-24-1939/
https://www.officialgazette.gov.ph/1940/01/22/manuel-l-quezon-sixth-state-of-the-nation-address-january-22-1940/
https://www.officialgazette.gov.ph/1941/01/31/manuel-l-quezon-seventh-state-of-the-nation-address-january-31-1941/
https://www.officialgazette.gov.ph/1945/06/09/sergio-osmena-state-of-the-nation-address-june-9-1945/
https://www.officialgazette.gov.ph/1946/06/03/president-roxas-on-first-state-of-the-nation-address-june-3-

In [ ]:
# # Pickle files for later use

 # Make a new directory to hold the text files
!mkdir transcripts

for i, c in enumerate(presidents):
    with open("transcripts/" + c + ".txt", "wb") as file:
        pickle.dump(transcripts[i], file)

In [ ]:
# Load pickled files
data = {}
for i, c in enumerate(presidents):
    with open("transcripts/" + c + ".txt", "rb") as file:
        data[c] = pickle.load(file)

In [ ]:
# We are going to change this to key: president, value: string format
def combine_text(list_of_text):
    '''Takes a list of text and combines them into one large chunk of text.'''
    combined_text = ' '.join(list_of_text)
    return combined_text

In [ ]:
# Combine it!
data_combined = {key: [combine_text(value)] for (key, value) in data.items()}

In [ ]:
# We can either keep it in dictionary format or put it into a pandas dataframe
import pandas as pd
pd.set_option('max_colwidth',150)

data_df = pd.DataFrame.from_dict(data_combined).transpose()
data_df.columns = ['transcript']
data_df

,transcript
Quezon-1935,Message\nof\nHis Excellency Manuel L. Quezon \nPresident of the Philippines\nTo the First National Assembly on National Defense [Delivered in the ...
Quezon-1936,Message\nof\nHis Excellency Manuel L. Quezon\nPresident of the Philippines\nTo the First National Assembly\nOn the Country’s conditions and proble...
Quezon-1937,Message\nof\nHis Excellency Manuel L. Quezon\nPresident of the Philippines\nTo the First National Assembly\nOn Improvement of Philippine Condition...
Quezon-1938,Message\nof\nHis Excellency Manuel L. Quezon\nPresident of the Philippines\nTo the First National Assembly\nOn the Revision of the System of Taxat...
Quezon-1939,Message\nof\nHis Excellency Manuel L. Quezon\nPresident of the Philippines\nTo the Second National Assembly\nOn the the State of the Nation and Im...
Quezon-1940,Message\nof\nHis Excellency Manuel L. Quezon\nPresident of the Philippines\nTo the Second National Assembly\nOn the The State of the Nation [Deliv...
Quezon-1941,Message\nof\nHis Excellency Manuel L. Quezon\nPresident of the Philippines\nTo the Second National Assembly\nOn the State of the Nation [Delivered...
Osmeña-1945,"Message\nof\nHis Excellency Sergio Osmeña\nPresident of the Philippines\nTo the Congress\nOn the State of the Nation [Delivered on June 9, 1945] G..."
Roxas-1946,"Message\nof\nHis Excellency Manuel Roxas\nPresident of the Philippines\nTo the Second Congress\nOn the State of the Nation [Released on June 3, 19..."
Roxas-1947,Message\nof\nHis Excellency Manuel Roxas\nPresident of the Philippines\nTo the First Congress of the Republic during its second session\nOn the St...
